# Tests and sanity checks of HMM implementations

For basic functionality tests of our HMM implementations, we use a simple Markov chain setup based on a coin-flipping task.

Consider a coin-flipping task where there are 2 states: 1) we are holding an unbiased coin and 2) we are holding a biased coin. Between coin flips, there is a probability p that we switch coins. This gives a Markov chain with a 2x2 transition matrix. The unbiased coin state has index 0 and the biased coin state has index 1.
    
There are two possible observations/emissions from each coin flip. We can either observe: 1) heads or 2) tails. The probability of observing heads/tails from the unbiased coin state is 0.5, and the probability from the biased coin state is q/1-q, for some probability q. The heads emission has index 0 and the tails emission has index 1.

In [1]:
import os
import numpy as np

In [2]:
def make_transition_matrix(p):
    return np.array([[p, 1-p], [1-p, p]])

def make_emission_matrix(q):
    return np.array([[0.5, 0.5], [q, 1-q]])

def draw_sequence(A, B, prior, T):
    num_states, num_obs = B.shape
    state = np.random.choice(num_states, 1, p=prior).item()
    emissions = [np.random.choice(num_obs, 1, p=B[state].flatten()).item()]
    for t in range(1, T):
        state = np.random.choice(num_states, 1, p=A[state].flatten()).item()
        emissions.append(np.random.choice(num_obs, 1, p=B[state].flatten()).item())
    return emissions

## Multinomial HMM based on 1st order Markov chain


In [6]:
from uncertainty_motion_prediction.predictor import HMMMultinomialFirstOrder

We test the forward/backward algorithms used for estimating the likelihood of a given sequence of emissions/observations.

In [12]:
p = 0.5
q = 0.5
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [13]:
hmm1 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm1.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq1 = np.array([0, 0, 1, 0])
prob1 = hmm1.get_sequence_likelihood_backward(test_seq1)
prob2 = hmm1.get_sequence_likelihood(test_seq1)

print(prob1, prob2)

0.0625 0.06250000000000003


We test the decoding algorithms used to estimate the most likely sequence of hidden states that produced the sequence of emissions/observations in our data. The decoding algorithm implemented is the Viterbi algorithm.

In [14]:
p = 0.5
q = 0.999 # Use 0.999 instead of 1, because I have yet to handle the case of np.log(0) in log-sum-exp function.
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [15]:
hmm2 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm2.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq2 = np.array([1, 0, 0])
path, prob = hmm2.decode(test_seq2)
print(path)
print(prob)

[0, 1, 1]
0.06237506250000003


Test the estimation of HMM model parameters from data. This uses the specialised form of the EM algorithm, the Baum-Welch algorithm. The algorithm can be prone to local minima, and may not converge well when all the parameters to be estimated are not well-initialised. You can test its robustness to initialisation but selectively initialising some of the parameters using the actual values.

In [ ]:
p = 0.5
q = 0.999 # Use 0.999 instead of 1, because I have yet to handle the case of np.log(0) in log-sum-exp function.
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [19]:
data = np.array([draw_sequence(transition_matrix, emission_matrix, prior, 10) for i in range(500)])
print(data)

# Don't give the correct values for any of the parameters. Use random initialisation.
hmm3 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm3.estimate_parameters(data)

print("Transition")
print(np.exp(hmm3._log_A))
print("Emission")
print(np.exp(hmm3._log_B))
print("Prior")
print(np.exp(hmm3._log_phi))

[[0 0 0 ... 1 1 0]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
Estimating HMM model parameters...
Iter 1, log-likelihood loss: -7.024722607974975, delta: inf
Iter 2, log-likelihood loss: -5.664936102659883, delta: 1.3597865053150917
Iter 3, log-likelihood loss: -5.66131013548803, delta: 0.0036259671718532616
Iter 4, log-likelihood loss: -5.658889304612112, delta: 0.0024208308759181563
Iter 5, log-likelihood loss: -5.657255492287608, delta: 0.0016338123245036584
Iter 6, log-likelihood loss: -5.656145098736711, delta: 0.0011103935508973706
Iter 7, log-likelihood loss: -5.655387159901618, delta: 0.0007579388350924532
Iter 8, log-likelihood loss: -5.65486849272569, delta: 0.0005186671759283001
Iter 9, log-likelihood loss: -5.654513089781826, delta: 0.0003554029438639361
Iter 10, log-likelihood loss: -5.654269421616268, delta: 0.00024366816555776438
Iter 11, log-likelihood loss: -5.654102342983989, delta: 0.00016707863227960473
Iter 1

In [18]:
data = np.array([draw_sequence(transition_matrix, emission_matrix, prior, 10) for i in range(500)])
print(data)

# Initialise the transition and prior models using correct values, and try to estimate the emission probabilities.
hmm3 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm3._initialise_transition_model(transition_matrix)
hmm3._initialise_prior_distribution(prior)
hmm3.estimate_parameters(data)

print("Transition")
print(np.exp(hmm3._log_A))
print("Emission")
print(np.exp(hmm3._log_B))
print("Prior")
print(np.exp(hmm3._log_phi))

[[0 0 0 ... 1 1 0]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
Estimating HMM model parameters...
Iter 1, log-likelihood loss: -7.3160232371448695, delta: inf
Iter 2, log-likelihood loss: -5.653891440391395, delta: 1.6621317967534743
Iter 3, log-likelihood loss: -5.653879230677302, delta: 1.220971409310323e-05
Iter 4, log-likelihood loss: -5.653867394295645, delta: 1.1836381657026607e-05
Iter 5, log-likelihood loss: -5.653855915128039, delta: 1.1479167605799034e-05
Iter 6, log-likelihood loss: -5.653844777992499, delta: 1.113713554001805e-05
Iter 7, log-likelihood loss: -5.653833968573029, delta: 1.0809419470625414e-05
Iter 8, log-likelihood loss: -5.653823473355083, delta: 1.0495217945383217e-05
Iter 9, log-likelihood loss: -5.65381327956661, delta: 1.0193788472889764e-05
Iter 10, log-likelihood loss: -5.653803375123801, delta: 9.9044428090167e-06
Transition
[[0.45704322 0.54295678]
 [0.45960241 0.54039759]]
Emission
[[0.6838772